<a href="https://colab.research.google.com/github/mbashish007/CommicSummarizer/blob/main/CustomBART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install -U "huggingface_hub[cli]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Apr 21 07:24:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             42W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
from torch.amp import autocast, GradScaler
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import get_scheduler
from datasets import load_dataset
import random
import os
import time
import wandb
import re


# Set device explicitly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




In [ ]:
# Force garbage collection and clear CUDA cache
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
device

device(type='cuda')

In [ ]:
# Initialize Weights & Biases
# run = wandb.init(project="custom-bart-4x4", config={
#     "model": "CustomBART",
#     "layers": 4,
#     "heads": 4,
#     "ff_dim": 1024,
#     "d_model": 512,
#     "max_len": 768,
#     "batch_size": 8,
#     "lr": 1e-4,
#     "grad_accum_steps": 4,
#     "total_steps": 250000  # Custom training for 250k steps
# })

# print("WANDB RUN ID:", run.id)

class LearnedPositionalEncoding(nn.Module):
    def __init__(self, max_len, d_model):
        super().__init__()
        self.pos_embedding = nn.Embedding(max_len, d_model)

    def forward(self, x):
        seq_len = x.size(1)
        positions = torch.arange(0, seq_len, dtype=torch.long, device=x.device)
        positions = positions.unsqueeze(0).expand(x.size(0), seq_len)
        return x + self.pos_embedding(positions)

class CustomEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward):
        super().__init__()
        self.layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward,  batch_first=True)

    def forward(self, src, src_mask=None, src_key_padding_mask=None ):
        return self.layer(src, src_mask=src_mask, src_key_padding_mask=src_key_padding_mask)

class CustomDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward):
        super().__init__()
        self.layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward, batch_first=True)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        return self.layer(tgt, memory, tgt_mask, memory_mask, tgt_key_padding_mask, memory_key_padding_mask)

class CustomBART(nn.Module):
    def __init__(self, vocab_size, d_model=512, nhead=4, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=1024, max_len=768):
        super().__init__()
        self.nhead = nhead
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_encoding = LearnedPositionalEncoding(max_len, d_model)

        self.encoder_layers = nn.ModuleList([CustomEncoderLayer(d_model, nhead, dim_feedforward) for _ in range(num_encoder_layers)])
        self.decoder_layers = nn.ModuleList([CustomDecoderLayer(d_model, nhead, dim_feedforward) for _ in range(num_decoder_layers)])

        self.output_proj = nn.Linear(d_model, vocab_size)


    def forward(self, input_ids, decoder_input_ids, attention_mask=None, decoder_attention_mask=None):

        # Convert and reshape masks
        if attention_mask is not None:
            attention_mask = attention_mask.bool()
        if decoder_attention_mask is not None:
            decoder_attention_mask = decoder_attention_mask.bool()

        #make attention mask for decoder
        if decoder_attention_mask is None and attention_mask is not None:
            decoder_attention_mask = ~attention_mask



        # Embeddings
        src_emb = self.position_encoding(self.token_embedding(input_ids))
        tgt_emb = self.position_encoding(self.token_embedding(decoder_input_ids))

        # Encoder
        memory = src_emb
        for layer in self.encoder_layers:
            memory = layer(memory, src_key_padding_mask= attention_mask)

        # Generate causal mask for decoder
            seq_len = decoder_input_ids.size(1)
            causal_mask = torch.triu(torch.full((seq_len, seq_len), float('-inf')), diagonal=1).to(input_ids.device)

        # Decoder
        output = tgt_emb
        for layer in self.decoder_layers:
            output = layer(
                tgt=output,
                memory=memory,
                tgt_mask=causal_mask,
                tgt_key_padding_mask=decoder_attention_mask,
                memory_key_padding_mask=attention_mask
            )

        return self.output_proj(output)



In [ ]:


# Initialize
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Config
class Config:
    batch_size = 16
    seq_len = 768
    mask_prob = 0.15
    min_text_length = 140
    buffer_size = 1000
    grad_accum_steps = 6
    save_every = 500
    label_smoothing = 0.1

config = Config()



# Text cleaning
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    # text = text.replace('`', '')
    return text if len(text) >= config.min_text_length else None

# Vectorized masking
def mask_tokens(input_ids, tokenizer):
    labels = input_ids.clone()
    special_tokens = torch.tensor(list(tokenizer.all_special_ids), device=input_ids.device)

    mask = (torch.rand(input_ids.shape, device=input_ids.device) < config.mask_prob)
    mask &= ~torch.isin(input_ids, special_tokens)
    mask &= (input_ids != tokenizer.pad_token_id)

    masked_input = input_ids.clone()
    masked_input[mask] = tokenizer.mask_token_id
    labels[~mask] = -100
    return masked_input, labels

def bart_mask_tokens(input_ids, tokenizer, mask_prob=config.mask_prob, max_span_len=2):
    special_ids = set(tokenizer.all_special_ids)
    pad_id = tokenizer.pad_token_id
    mask_id = tokenizer.mask_token_id

    input_ids = input_ids.clone()
    labels = input_ids.clone()

    for i in range(input_ids.size(0)):
        num_tokens = (input_ids[i] != pad_id).sum().item()
        num_to_mask = int(num_tokens * mask_prob)
        masked = 0
        attempts = 0

        while masked < num_to_mask and attempts < num_tokens * 2:
            span_len = random.randint(1, max_span_len)
            start = random.randint(1, num_tokens - span_len - 1)
            span = input_ids[i, start:start + span_len]

            if any(tok.item() in special_ids for tok in span):
                attempts += 1
                continue

            input_ids[i, start:start + span_len] = mask_id
            masked += span_len
            attempts += 1

    return input_ids, labels

class BookCorpusStream:


    def __init__(self, dataset_name=None, datafiles=None):
        # Load full dataset into memory (or keep as a reference to iterate)
        if dataset_name:
            self.dataset = load_dataset(dataset_name, split="train", trust_remote_code=True)
        else :
            self.dataset = load_dataset("parquet", split="train", data_files=datafiles,  trust_remote_code=True)
        # self.dataset = load_dataset(dataset_name, split="train", trust_remote_code=True)
        self.shuffled = self.dataset.shuffle(seed=41)
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        texts = []
        while len(texts) < config.batch_size:
            if self.index >= len(self.shuffled):
                if len(texts) == 0:
                    raise StopIteration
                break
            example = self.shuffled[self.index]
            self.index += 1
            cleaned = clean_text(example['text'])
            if cleaned:
                texts.append(cleaned)

        batch = tokenizer(
            texts,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=config.seq_len
        )

        # masked_input, labels = mask_tokens(batch['input_ids'], tokenizer)
        masked_input, labels = bart_mask_tokens(batch['input_ids'], tokenizer)


        decoder_input = batch['input_ids'].clone()
        decoder_input[:, 1:] = decoder_input[:, :-1].clone()
        decoder_input[:, 0] = tokenizer.bos_token_id

        return {
            'masked_input': masked_input.to(device),
            'decoder_input': decoder_input.to(device),
            'labels': labels.to(device),
            'attention_mask': batch['attention_mask'].to(device)
        }







/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [ ]:
# data_stream = BookCorpusStream(dataset_name="mintujupally/ROCStories")
# batch = next(data_stream)
# print(batch)

In [ ]:
device.type

'cuda'

In [ ]:
# Training loop

def skip_n_samples(dataset_iter, n):
    for _ in range(n):
        next(dataset_iter)
    return dataset_iter

def train(resume_from=0, wandb_run_id=None):
    wandb.init(
        project="custom-bart-4x4",
        resume="allow",
        id=wandb_run_id
    )
    wandb.watch(model, log="all")
    os.makedirs("checkpoints", exist_ok=True)

    # data_stream = BookCorpusStream()
    data_stream = BookCorpusStream(dataset_name="mintujupally/ROCStories")
    # data_stream.iter = skip_n_samples(data_stream.iter, resume_from)
    data_stream.index = resume_from  # Just update the index directly

    print("Starting Training")
    count = 0
    data_stream_counter = resume_from
    for global_step in range(50001):
        optimizer.zero_grad()

        for stp in range(config.grad_accum_steps):
            try:
                batch = next(data_stream)
                # print(batch)
            except StopIteration:
                data_stream = BookCorpusStream(dataset_name="mintujupally/ROCStories")
                batch = next(data_stream)
                # print(batch)

            data_stream_counter = data_stream_counter + config.batch_size
            with autocast(device_type=device.type, dtype=torch.float16):
                outputs = model(
                    input_ids=batch['masked_input'],
                    decoder_input_ids=batch['decoder_input'],
                    attention_mask=batch['attention_mask']
                )
                count = count+1
                if count % 500 == 0:
                    print(f"global step {global_step}")
                    predicted_ids = outputs.argmax(dim=-1)

                    masked_input_text = tokenizer.batch_decode(batch['masked_input'], skip_special_tokens=False)
                    decoder_input_text = tokenizer.batch_decode(batch['decoder_input'], skip_special_tokens=False)
                    predicted_text = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

                    labels_for_decode = batch['labels'].clone()
                    labels_for_decode[labels_for_decode == -100] = tokenizer.pad_token_id
                    target_text = tokenizer.batch_decode(labels_for_decode, skip_special_tokens=True)

                    print("\n [Visualization]")
                    print(f" Input     : {masked_input_text[0]}")
                    print(f" Target    : {target_text[0]}")
                    print(f" Decoder Input : {decoder_input_text[0]}")
                    print(f" Predicted : {predicted_text[0]}")

                labels = batch['labels'].view(-1)
                labels = labels.clone()
                labels[labels == tokenizer.pad_token_id] = -100

                loss = loss_fn(
                    outputs.view(-1, tokenizer.vocab_size),
                    labels
                ) / config.grad_accum_steps




            scaler.scale(loss).backward()

        scaler.step(optimizer)
        scaler.update()

        # 🧹 GPU memory cleanup after the batch
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

        if global_step % config.save_every == 0:
            if global_step == 0:
                continue
            ckpt_path = f"/content/drive/MyDrive/Bart_weights/bart_step_{global_step}_{int(time.time())}.pt"
            torch.save({
                'step': global_step,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scaler_state_dict': scaler.state_dict(),
            }, ckpt_path)
            wandb.save(ckpt_path)

        wandb.log({"data_stream_counter :": data_stream_counter})
        wandb.log({"loss": loss.item(), "step": global_step})

    wandb.finish()


In [ ]:
# Model and training setup
model = CustomBART(tokenizer.vocab_size).to(device)
model = torch.compile(model)
scaler = GradScaler()

#for xsum
learning_rate = 5e-5
num_epochs = 3
warmup_steps = 1000
batch_size = 16

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

loss_fn = nn.CrossEntropyLoss(ignore_index=-100, label_smoothing=0.1).to(device)




# Dataset size
num_training_samples = 204000  # or use estimated value
total_steps = (num_training_samples // (config.batch_size*config.grad_accum_steps)) * num_epochs

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

In [ ]:
# Step 2: Load the checkpoint
checkpoint = torch.load("/content/drive/MyDrive/Bart_weights/xsum_step_3500_1745219757.pt", map_location=device)
import copy
# Step 3: Restore everything
model.load_state_dict(checkpoint['model_state_dict'], strict=False)


optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scaler.load_state_dict(checkpoint['scaler_state_dict'])
lr_scheduler.load_state_dict(checkpoint['lr_scheduler_state_dict'])
global_step = checkpoint['step']



In [ ]:
print(tokenizer.pad_token_id)

1


In [ ]:
# # Manually override LR / weight decay if needed
# for param_group in optimizer.param_groups:
#     param_group['lr'] = 8e-5
#     # param_group['weight_decay'] = 0.01

In [ ]:
for param_group in optimizer.param_groups:
    print(param_group['lr'], param_group['weight_decay'])

2.6725581395348835e-05 0.01


In [ ]:
# train(resume_from= 6657104,wandb_run_id='lmyky6ou')

In [ ]:
def distillLearn(resume_from=0, wandb_run_id=None, temperature=1, alpha =0.5, dataset_name=None, datafiles=None):
    wandb.init(
        project="custom-bart-4x4",
        resume="allow",
        id=wandb_run_id
    )
    wandb.watch(model, log="all")
    os.makedirs("checkpoints", exist_ok=True)

    if dataset_name:
        data_stream = BookCorpusStream(dataset_name=dataset_name)
    else :
        data_stream = BookCorpusStream(datafiles=datafiles)
    # data_stream = BookCorpusStream(dataset_name=dataset_name)
    # data_stream.iter = skip_n_samples(data_stream.iter, resume_from)
    data_stream.index = resume_from  # Just update the index directly

    print("Starting Training")
    count = 0
    data_stream_counter = resume_from
    for global_step in range(50001):
        optimizer.zero_grad()

        for stp in range(config.grad_accum_steps):
            try:
                batch = next(data_stream)
                # print(batch)
            except StopIteration:
                # data_stream = BookCorpusStream(dataset_name=dataset_name)
                # batch = next(data_stream)
                # print(batch)
                print("End of DataSet")
                ckpt_path = f"/content/drive/MyDrive/Bart_weights/bart_step_{global_step}_{int(time.time())}.pt"
                torch.save({
                    'step': global_step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scaler_state_dict': scaler.state_dict(),
                }, ckpt_path)
                wandb.save(ckpt_path)
                return

            data_stream_counter = data_stream_counter + config.batch_size
            with autocast(device_type=device.type, dtype=torch.float16):

                                # Student forward
                student_logits = model(
                    input_ids=batch['masked_input'],
                    decoder_input_ids=batch['decoder_input'],
                    attention_mask=batch['attention_mask']
                )  # (B, T, V)

                # Teacher forward (no grad)
                with torch.no_grad():
                    teacher_outputs = teacher_model(
                        input_ids=batch['masked_input'],
                        decoder_input_ids=batch['decoder_input'],
                        attention_mask=batch['attention_mask']
                    )
                    teacher_logits = teacher_outputs.logits  # (B, T, V)
                    teacher_predicted_ids = teacher_logits.argmax(dim=-1)

                labels = batch['labels'].view(-1)
                labels[labels == tokenizer.pad_token_id] = -100

                # Losses
                student_logits_flat = student_logits.view(-1, tokenizer.vocab_size)
                teacher_logits_flat = teacher_logits.view(-1, tokenizer.vocab_size)

                ce_loss = ce_loss_fn(student_logits_flat, labels)

                # Soft targets: log_softmax from student, softmax from teacher
                student_log_probs = F.log_softmax(student_logits_flat / temperature, dim=-1)
                teacher_probs = F.softmax(teacher_logits_flat / temperature, dim=-1)
                kl_loss = kl_loss_fn(student_log_probs, teacher_probs) * (temperature ** 2)

                loss = (alpha * kl_loss + (1 - alpha) * ce_loss) / config.grad_accum_steps
                count = count+1
                if count % 250 == 0:
                    print(f"global step {global_step}")
                    predicted_ids = student_logits.argmax(dim=-1)

                    masked_input_text = tokenizer.batch_decode(batch['masked_input'], skip_special_tokens=False)
                    decoder_input_text = tokenizer.batch_decode(batch['decoder_input'], skip_special_tokens=False)
                    predicted_text = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

                    labels_for_decode = batch['labels'].clone()
                    labels_for_decode[labels_for_decode == -100] = tokenizer.pad_token_id
                    target_text = tokenizer.batch_decode(labels_for_decode, skip_special_tokens=True)

                    # Decode the teacher's predictions
                    teacher_predicted_text = tokenizer.batch_decode(teacher_predicted_ids, skip_special_tokens=True)


                    print("\n🔍 [Visualization]")
                    print(f" Input     : {masked_input_text[0]}")
                    print(f" Target    : {target_text[0]}")
                    print(f" Decoder Input : {decoder_input_text[0]}")
                    print(f" Predicted : {predicted_text[0]}")
                                        # Visualization
                    print("\n [Teacher Visualization]")
                    print(f" Teacher Predicted : {teacher_predicted_text[0]}")



            scaler.scale(loss).backward()

        scaler.step(optimizer)
        scaler.update()

        # 🧹 GPU memory cleanup after the batch
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

        if global_step % config.save_every == 0:
            if global_step == 0:
                continue
            ckpt_path = f"/content/drive/MyDrive/Bart_weights/bart_step_{global_step}_{int(time.time())}.pt"
            torch.save({
                'step': global_step,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scaler_state_dict': scaler.state_dict(),
            }, ckpt_path)
            wandb.save(ckpt_path)

        wandb.log({"data_stream_counter :": data_stream_counter})
        # wandb.log({"loss": loss.item(), "step": global_step})
        wandb.log({
              "loss/total": loss.item(),
              "loss/batch": loss.item()*config.grad_accum_steps,
              "loss/ce": ce_loss.item(),
              "loss/kl": kl_loss.item(),
              "step": global_step
          })

    wandb.finish()


In [ ]:
# !huggingface-cli login

In [ ]:
## Run on mini-en
# distillLearn(resume_from=	0, wandb_run_id='9fxm8pu9', temperature=1.5, alpha=0.4, dataset_name="nampdn-ai/mini-en")

In [ ]:
## Run on mini-en
base_url = "https://huggingface.co/datasets/jxie/wiki_books2048/resolve/main/data/"
wikibooks2048 = {
    "train": [
            # base_url+"train-00000-of-00045-f1d34d1cf75dedb6.parquet",
            # base_url+"train-00001-of-00045-4971490a8ce72a07.parquet",
            # base_url+"train-00002-of-00045-b2a95d8c5d4fa373.parquet",
            # base_url+"train-00003-of-00045-fdb37b478ce7aad6.parquet",
            # base_url+"train-00004-of-00045-3e0334dd7e5545b0.parquet",
            # base_url+"train-00005-of-00045-74fbe165082d0a23.parquet",
            # base_url+"train-00006-of-00045-d8b3a885b00e91ff.parquet",
            # base_url+"train-00007-of-00045-2739109991368738.parquet",
            # base_url+"train-00008-of-00045-f2ae50cd310f46b0.parquet",
            # base_url+"train-00009-of-00045-8cb0eb080c365907.parquet",
            base_url+"train-00010-of-00045-2da73ba6a406b6e3.parquet",
            base_url+"train-00011-of-00045-25aa4ffd431fba04.parquet",
            base_url+"train-00012-of-00045-62f4d70474100212.parquet",
            base_url+"train-00013-of-00045-7d7f0f479108ae63.parquet",
            base_url+"train-00014-of-00045-67501023d35af1d9.parquet",
            base_url+"train-00015-of-00045-e0f57e81496a76d3.parquet",
            base_url+"train-00016-of-00045-9ee7fa32efb6686a.parquet",
            base_url+"train-00017-of-00045-28039a49583960de.parquet",
            base_url+"train-00018-of-00045-0b6de236903bf102.parquet",
            base_url+"train-00019-of-00045-91465e94cccb4d61.parquet"
    ]
}

# distillLearn(resume_from=	277920, wandb_run_id='wgbgmcdr', temperature=1.6, alpha=0.36, datafiles=wikibooks2048)


In [ ]:
class XSumStream:
    def __init__(self, tokenizer, seed=45, dataset_name=None , streaming=False):
        self.tokenizer = tokenizer
        self.config = config

        self.dataset = load_dataset(dataset_name, split="train", streaming=streaming)
        if not streaming:
            self.dataset = self.dataset.shuffle(seed=seed)
            self.index = 0
        else:
            self.iter = iter(self.dataset)

    def __iter__(self):
        return self

    def __next__(self):
        inputs, targets = [], []

        while len(inputs) < self.config.batch_size:

          if hasattr(self, 'iter'):
              example = next(self.iter)
          else:
              if self.index >= len(self.dataset):
                  if len(inputs) == 0:
                      raise StopIteration
                  break
              example = self.dataset[self.index]
              self.index += 1

          source = "summarize: " + example["document"].strip()
          target = example["summary"].strip()

          if source and target:
              inputs.append(source)
              targets.append(target)



        enc = self.tokenizer(
            inputs,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=self.config.seq_len
        )

        dec = self.tokenizer(
            targets,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=self.config.seq_len
        )

        decoder_input_ids = dec['input_ids'].clone()
        decoder_input_ids[:, 1:] = decoder_input_ids[:, :-1].clone()
        decoder_input_ids[:, 0] = self.tokenizer.bos_token_id

        return {
            'encoder_input': enc['input_ids'].to(device),
            'decoder_input': decoder_input_ids.to(device),
            'labels': dec['input_ids'].to(device),
            'attention_mask': enc['attention_mask'].to(device)
        }

In [ ]:
def distillLearnXsum(resume_from=0,resume_step =0, total_steps = 0, wandb_run_id=None, tokenizer=None, teacher_model = None, temperature=1, alpha =0.5, dataset_name=None, datafiles=None):
    wandb.init(
        project="custom-bart-4x4",
        resume="allow",
        id=wandb_run_id
    )
    wandb.watch(model, log="all")
    os.makedirs("checkpoints", exist_ok=True)

    if dataset_name:
        data_stream = XSumStream(tokenizer, dataset_name=dataset_name)
    else :
        data_stream = XSumStream(tokenizer, datafiles=datafiles)
    # data_stream = BookCorpusStream(dataset_name=dataset_name)
    # data_stream.iter = skip_n_samples(data_stream.iter, resume_from)
    data_stream.index = resume_from  # Just update the index directly

    print("Starting Training")
    count = 0
    data_stream_counter = resume_from
    for global_step in range(resume_step, total_steps+2):
        optimizer.zero_grad()

        for stp in range(config.grad_accum_steps):
            try:
                batch = next(data_stream)
                # print(batch)
            except StopIteration:
                seed = random.randint(40, 50)
                print("new seed:", seed)
                data_stream_counter = 0
                data_stream = XSumStream(tokenizer, seed=seed, dataset_name=dataset_name)
                batch = next(data_stream)
                # print(batch)
                # print("End of DataSet")
                # ckpt_path = f"/content/drive/MyDrive/Bart_weights/xsum_step_{global_step}_{int(time.time())}.pt"
                # torch.save({
                #     'step': global_step,
                #     'model_state_dict': model.state_dict(),
                #     'optimizer_state_dict': optimizer.state_dict(),
                #     'scaler_state_dict': scaler.state_dict(),
                #     'lr_scheduler_state_dict': lr_scheduler.state_dict()
                # }, ckpt_path)
                # wandb.save(ckpt_path)
                # return

            data_stream_counter = data_stream_counter + config.batch_size
            with autocast(device_type=device.type, dtype=torch.float16):


                                # Student forward
                student_logits = model(
                    input_ids=batch['encoder_input'],
                    decoder_input_ids=batch['decoder_input'],
                    attention_mask=batch['attention_mask']
                )  # (B, T, V)

                # Teacher forward (no grad)
                with torch.no_grad():
                    teacher_outputs = teacher_model(
                        input_ids=batch['encoder_input'],
                        decoder_input_ids=batch['decoder_input'],
                        attention_mask=batch['attention_mask']
                    )
                    teacher_logits = teacher_outputs.logits  # (B, T, V)
                    teacher_predicted_ids = teacher_logits.argmax(dim=-1)

                labels = batch['labels'].view(-1)
                labels[labels == tokenizer.pad_token_id] = -100



                # Align teacher logits shape
                if teacher_logits.size(-1) != tokenizer.vocab_size:
                    teacher_logits = F.pad(teacher_logits, (0, tokenizer.vocab_size - teacher_logits.size(-1)), "constant", 0)

                teacher_logits_flat = teacher_logits.view(-1, tokenizer.vocab_size)
                student_logits_flat = student_logits.view(-1, tokenizer.vocab_size)


                ce_loss = ce_loss_fn(student_logits_flat, labels)

                # Soft targets: log_softmax from student, softmax from teacher
                student_log_probs = F.log_softmax(student_logits_flat / temperature, dim=-1)
                teacher_probs = F.softmax(teacher_logits_flat / temperature, dim=-1)
                kl_loss = kl_loss_fn(student_log_probs, teacher_probs) * (temperature ** 2)

                loss = (alpha * kl_loss + (1 - alpha) * ce_loss) / config.grad_accum_steps

                if(global_step >=500 and global_step < 750):
                   alpha = 0.4
                elif (global_step >=750 and global_step < 1000):
                   alpha = 0.3
                elif (global_step >=1000 and global_step < 1500):
                   alpha = 0.2
                else:
                   alpha = 0.1

                count = count+1
                if count % 250 == 0:
                    print(f"global step {global_step}")
                    predicted_ids = student_logits.argmax(dim=-1)

                    encoder_input_text = tokenizer.batch_decode(batch['encoder_input'], skip_special_tokens=False)
                    decoder_input_text = tokenizer.batch_decode(batch['decoder_input'], skip_special_tokens=False)
                    predicted_text = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

                    labels_for_decode = batch['labels'].clone()
                    labels_for_decode[labels_for_decode == -100] = tokenizer.pad_token_id
                    target_text = tokenizer.batch_decode(labels_for_decode, skip_special_tokens=True)

                    # Decode the teacher's predictions
                    teacher_predicted_text = tokenizer.batch_decode(teacher_predicted_ids, skip_special_tokens=True)


                    print("\n [Visualization]")
                    print(f" Input     : {encoder_input_text[0]}")
                    print(f" Target    : {target_text[0]}")
                    print(f" Decoder Input : {decoder_input_text[0]}")
                    print(f" Predicted : {predicted_text[0]}")
                                        # Visualization
                    print("\n🔍 [Teacher Visualization]")
                    print(f" Teacher Predicted : {teacher_predicted_text[0]}")



            scaler.scale(loss).backward()

        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        #  GPU memory cleanup after the batch
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

        if global_step % config.save_every == 0:
            if global_step == 0:
                continue
            ckpt_path = f"/content/drive/MyDrive/Bart_weights/xsum_step_{global_step}_{int(time.time())}.pt"
            torch.save({
                'step': global_step,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scaler_state_dict': scaler.state_dict(),
                'lr_scheduler_state_dict': lr_scheduler.state_dict(),
            }, ckpt_path)
            wandb.save(ckpt_path)

        wandb.log({"data_stream_counter :": data_stream_counter})
        # wandb.log({"loss": loss.item(), "step": global_step})
        wandb.log({
              "loss/total": loss.item(),
              "loss/batch": loss.item()*config.grad_accum_steps,
              "loss/ce": ce_loss.item(),
              "loss/kl": kl_loss.item(),
              "step": global_step,
              "lr": lr_scheduler.get_last_lr()[0]
          })

    wandb.finish()
    ckpt_path = f"/content/drive/MyDrive/Bart_weights/xsum_step_{int(time.time())}.pt"
    torch.save({
        'step': global_step,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
        'lr_scheduler_state_dict': lr_scheduler.state_dict(),
    }, ckpt_path)


In [ ]:
teacher_model_xsum = BartForConditionalGeneration.from_pretrained("facebook/bart-large-xsum").to(device)
teacher_model_xsum.eval()  # Set to eval mode

# Losses
ce_loss_fn = nn.CrossEntropyLoss(ignore_index=-100, label_smoothing=0.1)
kl_loss_fn = nn.KLDivLoss(reduction='batchmean')

# temperature = 1.5
# alpha = 0.4

distillLearnXsum(resume_from=	100000, resume_step = 3501, total_steps = total_steps, wandb_run_id='4nyi4xiu', tokenizer=tokenizer, teacher_model= teacher_model_xsum, temperature=2, alpha=0.5, dataset_name='EdinburghNLP/xsum')

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

global step 3542

🔍 [Visualization]
📝 Input     : <s>summarize: The joint venture will combine Tesco's 131 outlets in the country with CRE's almost 3,000 stores, called Vanguard.
CRE will own 80% of the new chain and Tesco will have a 20% stake.
"The partnership creates a strong platform in one of the world's largest markets," Tesco chief executive Philip Clarke said in a statement.
"We can now combine our strengths to build a profitable multichannel business, offering our customers in China the best of modern retail."
According to figures from research group IGD, China is the world's biggest market for food and groceries with an annual value of more than one trillion dollars.
And the market is forecast to grow by 50% over the next three years.
Tesco, the biggest supermarket chain in the UK, has been keen to tap into this potential, and has been operating in the country since 2004.
The firm has also been eyeing the Indian market - which is forecast to witness rapid growth in the coming

wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


global step 4042

🔍 [Visualization]
📝 Input     : <s>summarize: It follows a profit warning from the Fife-based company, issued last year, after a number of shops and financial sector customers deferred orders.
However, Havelock said a number of its divisions were seeing improvement.
In healthcare it said it had won its first two significant orders, which will be supplied mostly during 2015.
The firm which is headquartered in Dalgety Bay said in retail it was "developing new customer relationships from which we expect to benefit during the coming year".
And as expected it said education activity was "beginning to show signs of recovery" with an order book of Â£25m compared with Â£14m last year.
Last May, Havelock Europa announced the resignation of its group finance director. It said the search for a new chief executive was beginning.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


new seed: 43
global step 4625

🔍 [Visualization]
📝 Input     : <s>summarize: Bardsey Island Trust is looking for a new information and volunteer officer to spend 18 weeks living in the tiny community.
The job offers basic accommodation and the chance to work in a national nature reserve, home to dolphins and seals.
The island, located two miles off Llyn peninsula, is only accessible by private boat.
But Caroline Williams, administrator with the Bardsey Island Trust, warned the job is not a holiday.
"It's lonely and isolated, and you have to be prepared for that," she said. "But the people who live there love it.
"And it's not totally cut off - there is the internet and phone. It is the perfect job for someone who is at university or who has just graduated and is looking for some experience."
The island, called Ynys Enlli in Welsh, is limited to around 2,000 visitors per season, and has seven permanent residents.
It is said to be the burial ground of 20,000 saints, and was an important 